In [1]:
try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
except ModuleNotFoundError:
    pass

✨🍰✨ Everything looks OK!


In [2]:
import time
t1 = time.perf_counter()
try:
    import condacolab
    from google.colab import files
    from IPython.display import clear_output
    condacolab.check()
    !conda install -q -y -c conda-forge mdtraj openmm cudatoolkit=11.2 openmmforcefields openmm-plumed

    on_colab = True
    clear_output()             # clear the excessive installation outputs (disable incase of error check)
    print("Dependencies successfully installed and imported!")
except ModuleNotFoundError:
    on_colab = False

!pwd

# required for simulation with Plumed on gpu
from sys import stdout
from openmmplumed import PlumedForce
from openmm.app import *
from openmm import *
from openmm.unit import *

# required for analysis
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

t2 = time.perf_counter()
print('time taken to run:',t2-t1)


Dependencies successfully installed and imported!
/content
time taken to run: 66.27913272599972


In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/FKBP_openmm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
import time
from sys import stdout
import re

import warnings
warnings.filterwarnings("ignore")

from openmm import *
from openmm.app import *
from openmm.unit import *
from openmmplumed import PlumedForce
#import pdbfixer
import mdtraj as md
from mdtraj.reporters import XTCReporter
import json


In [8]:
def get_psf(path='openmm/'):

    file_path = path + 'sysinfo.dat'
    with open(file_path, 'r') as file:
        data = json.load(file)
    dim = data['dimensions']

    psf=CharmmPsfFile(path+'step3_input.psf')
    psf.setBox(dim[0]*angstroms,dim[0]*angstroms,dim[0]*angstroms)
    return psf

def get_pdb(path='openmm/'):
    return PDBFile(path+'step3_input.pdb')

In [9]:
def get_params(path='toppar/'):
    prot_rtf=os.path.join(path,'top_all36_prot.rtf')
    prot_prm=os.path.join(path,'par_all36m_prot.prm')
    ligand_rtf=os.path.join(path,'top_all36_cgenff.rtf')
    ligand_prm=os.path.join(path,'par_all36_cgenff.prm')
    solvent_str=os.path.join(path,'toppar_water_ions.str')
    params = CharmmParameterSet(prot_rtf,ligand_rtf,prot_prm,ligand_prm,solvent_str)
    return params

In [10]:
def get_LangevinM_system(psf,params,temp=300,dt=0.002):
    '''
    Creates a simulation instance under Langevin Middle Integrator designed in openmm
    '''
    # CHARMM lipid forcefield was parameterized for 8-12 cutoff
    system = psf.createSystem(params,nonbondedMethod=PME,nonbondedCutoff=1.2*nanometer,\
                              switchDistance=1.0*nanometer,constraints=HBonds);
    integrator = LangevinMiddleIntegrator(temp*kelvin, 1/picoseconds,dt*picoseconds);
    platform = Platform.getPlatformByName('CUDA');
    simulation = Simulation(psf.topology, system, integrator, platform)
    return simulation

In [11]:
def add_pos_res(positions,top,simulation,k=10,molecule='protein'):
    '''

    Adds an harmonic potential to the heavy atoms of the system(proteins) with an user defined force constant 'k'

    '''

    AA=['ALA','ASP','CYS','GLU','PHE','GLY','HIS','ILE','LYS','LEU','MET','ARG','PRO','GLN','ASN','SER','THR','VAL','TRP','TYR']
    force = CustomExternalForce("kprot*periodicdistance(x, y, z, x0, y0, z0)^2") # Harmonic potential for position restrain
    force.addGlobalParameter("kprot",k*kilojoules_per_mole/angstroms**2)
    force.addPerParticleParameter("x0")
    force.addPerParticleParameter("y0")
    force.addPerParticleParameter("z0")

    index=0;
    for i, res in enumerate(top.residues()):
        if res.name in AA:                              # Required to select only the protein atoms
            for at in res.atoms():
                if not re.search(r'H',at.name):         # All heavy Atoms -(exculdes Hydrogens)
                    force.addParticle(index,positions[index].value_in_unit(nanometers))
                index+=1;

    posres_sys=simulation.context.getSystem()                  # A gets System for a simulation instance
    posres_sys.addForce(force)                          # Modifies system with custom Force

    if molecule.split('+')[-1]== 'ligand':
        memb_force = CustomExternalForce('klig*periodicdistance(x, y, z, x0, y0, z0)^2;')
        memb_force.addGlobalParameter('klig',k*kilojoules_per_mole/angstroms**2)
        memb_force.addPerParticleParameter('x0')
        memb_force.addPerParticleParameter('y0')
        memb_force.addPerParticleParameter('z0')
        topology=md.Topology.from_openmm(top)
        expression='resname DMS and name S or resname DMS and name O'
        python_exp=topology.select_expression(expression)
        req_indices=np.array(eval(python_exp))
        for ind in req_indices:
            memb_force.addParticle(ind,positions[ind].value_in_unit(nanometers))
        posres_sys.addForce(memb_force)

    simulation.context.reinitialize()                          # initializes the simulation instance with the modified system
    return simulation


In [12]:
def run_MD(simulation,positions,nsteps=50000,use_plumed=True,plumed_file=None,report_steps=True):


    pdb_positions = simulation.context.getState(getPositions=True,enforcePeriodicBox=True).getPositions()
    PDBFile.writeFile(simulation.topology, pdb_positions, open('test_prod_starting.pdb', 'w'))

    if use_plumed:
        if plumed_file is None:
            raise ValueError("PLUMED file must be provided when use_plumed is True.")
        fid=open(plumed_file,'r')
        ff=fid.read()
        force=PlumedForce(ff)
        pl_system=simulation.context.getSystem()
        pl_system.addForce(force)
        simulation.context.reinitialize(True)

    simulation.reporters=[]

    simulation.context.setPositions(positions)
    outfname=f'simulation_prod_run1.xtc'
    topology=md.Topology.from_openmm(simulation.topology)
    python_expression=topology.select_expression('not water and not name CLA and not name NA')
    req_indices=np.array(eval(python_expression))
    simulation.reporters.append(XTCReporter(outfname, 500, atomSubset=req_indices))

    if report_steps:
      simulation.reporters.append(StateDataReporter(stdout, 500,
                                                    step=True,
                                                    progress=True,
                                                    speed=True,
                                                    totalSteps=nsteps,
                                                    potentialEnergy=True,
                                                    kineticEnergy=True,
                                                    temperature=True))

    simulation.step(nsteps)

    pdb_positions = simulation.context.getState(getPositions=True,enforcePeriodicBox=True).getPositions()
    PDBFile.writeFile(simulation.topology, pdb_positions, open('test_prod_run1_end.pdb', 'w'))

    # Initialize PLUMED if specified
    if use_plumed:
        simulation.context.getSystem().removeForce(simulation.context.getSystem().getNumForces()-1)  #from plumedforc

In [15]:
pdb = get_pdb()
psf = get_psf()
params = get_params()

In [16]:
simulation = get_LangevinM_system(psf,params,temp=303,dt=0.002)
simulation = add_pos_res(pdb.positions,psf.topology,simulation,10,'protein+ligand')    # 10 Kj/(mol.A^2) `
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy(tolerance=500*kilojoule/(mole*nanometer), maxIterations=5000)

#minim_positions = simulation.context.getState(getPositions=True).getPositions()

In [17]:
def equilibrium_steps(pro_par,lig_par):
  #simulation_constraints
  simulation.context.setParameter('kprot',pro_par*kilojoules_per_mole/angstroms**2)
  simulation.context.setParameter('klig',lig_par*kilojoules_per_mole/angstroms**2)

  positions= simulation.context.getState(getPositions=True).getPositions()
  run_MD(simulation,positions,nsteps=500,use_plumed=False,plumed_file=None,report_steps=False)


In [18]:
equilibration = False

if equilibration:
  equilibration_parameter = [[2.0,2.0],[1.0,1.0],[0.2,0.5],[0.0,0.2],[0.0,0.0]]

  for i,param in enumerate(equilibration_parameter):
    print('===== Equilibration step %i ====='%(i+1))
    pro_par,lig_par = param[0],param[1]
    equilibrium_steps(pro_par,lig_par)

  simulation.reporters=[]

  positions= simulation.context.getState(getPositions=True).getPositions()
  run_MD(simulation,positions,nsteps=50000,use_plumed=False,plumed_file=None)

  pdb_positions = simulation.context.getState(getPositions=True,enforcePeriodicBox=True).getPositions()
  PDBFile.writeFile(simulation.topology, pdb_positions, open('equil_final.pdb', 'w'))
  simulation.saveState('equil_finate.state')
  simulation.saveCheckpoint('equil_chkptfile.chk')

else:
  print('===== Load pre-Equilibrated system Checkpoint file =====')
  chkptfile = 'equil_chkptfile.chk'
  simulation.loadCheckpoint(chkptfile)



===== Load pre-Equilibrated system Checkpoint file =====


In [ ]:
positions= simulation.context.getState(getPositions=True).getPositions()
run_MD(simulation,positions,nsteps=5000,use_plumed=True,plumed_file='plumed_initial_v1.dat')

#"Progress (%)","Step","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
5010.0%,250500,-367142.5922004408,72776.04548791493,304.080124423418,0
5020.0%,251000,-366436.2142707533,73434.3567535516,306.83074614547047,116
5030.0%,251500,-364106.9222785658,74761.56560741295,312.37622241739894,117
5040.0%,252000,-362888.0453254408,76275.52486325655,318.7019978260855,116
5050.0%,252500,-361379.54679028457,77132.56253706315,322.2829580271992,116
5060.0%,253000,-361363.3656379408,78212.88136161934,326.7968537793744,116
5070.0%,253500,-359963.4818488783,78843.64125441719,329.43235761030354,115
5080.0%,254000,-357992.6166145033,80154.85305654292,334.91099340677664,115


In [ ]:
outfname=f'production_run.xtc'
topology=md.Topology.from_openmm(simulation.topology)
python_expression=topology.select_expression('all')
req_indices=np.array(eval(python_expression))
simulation.reporters.append(XTCReporter(outfname, 500, atomSubset=req_indices))
